In [2]:
import requests
import pandas as pd
import requests
import numpy as np
import json

x = requests.get('https://ec.europa.eu/esco/api/search?language=en&type=skill&text=algorithms')
x = x.text
x = json.loads(x)
x = pd.json_normalize(x , record_path=['_embedded','results'])
x.drop(x.columns.difference(['className','uri','searchHit','title','broaderHierarchyConcept','_links.self.href','_links.self.uri','_links.self.title','broaderSkill']),axis = 1,inplace=True)
x.head()

,className,uri,searchHit,title,broaderHierarchyConcept,_links.self.href,_links.self.uri,_links.self.title,broaderSkill
0,Skill,http://data.europa.eu/esco/skill/54924a2c-daca...,algorithms,algorithms,[http://data.europa.eu/esco/isced-f/0612],https://ec.europa.eu/esco/api/resource/skill?u...,http://data.europa.eu/esco/skill/54924a2c-daca...,algorithms,NaN
1,Skill,http://data.europa.eu/esco/skill/52cf3037-ab53...,The techniques and principles of software deve...,Groovy,[http://data.europa.eu/esco/isced-f/0613],https://ec.europa.eu/esco/api/resource/skill?u...,http://data.europa.eu/esco/skill/52cf3037-ab53...,Groovy,[http://data.europa.eu/esco/skill/21d2f96d-35f...
2,Skill,http://data.europa.eu/esco/skill/d56fc2b5-4b0a...,The techniques and principles of software deve...,Scratch (computer programming),[http://data.europa.eu/esco/isced-f/0613],https://ec.europa.eu/esco/api/resource/skill?u...,http://data.europa.eu/esco/skill/d56fc2b5-4b0a...,Scratch (computer programming),[http://data.europa.eu/esco/skill/21d2f96d-35f...
3,Skill,http://data.europa.eu/esco/skill/a4d336a6-9ffd...,The techniques and principles of software deve...,ML (computer programming),[http://data.europa.eu/esco/isced-f/0613],https://ec.europa.eu/esco/api/resource/skill?u...,http://data.europa.eu/esco/skill/a4d336a6-9ffd...,ML (computer programming),[http://data.europa.eu/esco/skill/21d2f96d-35f...
4,Skill,http://data.europa.eu/esco/skill/47b9bbcf-356c...,The techniques and principles of software deve...,Assembly (computer programming),[http://data.europa.eu/esco/isced-f/0613],https://ec.europa.eu/esco/api/resource/skill?u...,http://data.europa.eu/esco/skill/47b9bbcf-356c...,Assembly (computer programming),[http://data.europa.eu/esco/skill/21d2f96d-35f...


In [3]:
certifications = pd.read_csv('certifications_final.csv', index_col=0)
certifications.drop(certifications.columns.difference(['cert_name','isco3']),axis=1,inplace=True)
classes = [251, 243, 214, 522, 242,334]#[121, 214, 216, 241, 242, 243, 251, 311, 411, 432] #[12, 21, 24, 25, 52]#[251, 243, 214, 522, 242,334]
certifications = certifications[certifications['isco3'].isin(classes)]
certifications.reset_index(drop=True, inplace=True)
certifications

,cert_name,isco3
0,HP ASE Service Manager Software Implementation v9,251
1,Certified Software Quality Engineer,251
2,AIS - HP Operations Orchestration v9,251
3,Brocade Accredited Campus Networking Specialist,251
4,AIS - HP BladeSystem Solutions [2010],251
...,...,...
735,Association for Healthcare Documentation Integ...,334
736,Registered Healthcare Documentation Specialist,334
737,Certified Healthcare Documentation Specialist,334
738,NAFA Certified Air Filter Specialist,522


In [10]:
df = pd.read_csv('escoe-knowledge_final.csv', index_col=0)
df
df[df['skill'].str.contains("cloud")==True]
#df.to_json('escoe_knowledge_final.json', orient='records', lines=True)

,skill,codes,index
10857,cloud technologies,[252],10857
10858,cloud computing,[252],10858


In [11]:
def wextract(text):
    '''
    extract skills from a given text (certification title/description) using WEX

    :param text: <str> text to be analyzed
    :return: <DataFrame> returns all the extracted skills by WEX
    '''
    response = requests.post('https://sortinghat-load-balancer-dev.eu-gb.mybluemix.net/skills/parse/text', data={"text": text})

    extracted_skills = pd.json_normalize(response.json(),record_path=['skills'])
    extracted_skills.drop(extracted_skills.columns.difference(['text', 'confidence']), axis=1, inplace = True)
    extracted_skills.rename(columns={'text':'skill'}, inplace=True)
    return extracted_skills

wextract('AIS - HP Operations Orchestration v9')	#'GIAC Secure Software Programmer - Java')

,skill,confidence
0,AIS,0.943305
1,HP Operations Orchestration v9,0.963428


In [12]:
import re

skills2escoe = pd.read_csv('escoe-knowledge_final.csv', index_col=0)

def match_skill(skill):
    '''
    search for skills in escoe taxonomy and retrieve the corresponding occupation code

    :param skill: <str> skill to be searched
    :return: <list> returns all the extracted occupation codes
    '''
    skill = skill.lower()
    if skills2escoe[skills2escoe['skill']==skill]['codes'].empty:
        return 0
    else:
        x = skills2escoe[skills2escoe['skill']==skill]['codes'].values[0]
        return x
    
p = match_skill('office devices')
p

'[524 332 333 243 132]'

In [13]:
from nltk.util import ngrams
from collections import Counter
from itertools import chain
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def string_match(string1, string2):
    '''
    compares two given strings and raises a flag if the 
    similarity is above a predefined threshold
    
    :param string1: <str> string 1 to be matched
    :param strign2: <str> string 2 to be matched
    :return: <int> returns the how many simalar words exist between the two strings
    '''
    matching_score = 0

    tokenized_text1 = string1.split(" ")
    tokenized_text2 = string2.split(" ")
    stemmed_text1 = [stemmer.stem(token) for token in tokenized_text1]
    stemmed_text2 = [stemmer.stem(token) for token in tokenized_text2]
  
    #n = 1
    #ngrams1= ngrams(stemmed_text1, n)
    #ngrams2= ngrams(stemmed_text2, n)                    
    
    counter1 = Counter(stemmed_text1)      #count occurrences of each n-gram
    counter2 = Counter(stemmed_text2)
    
    s = [k for k,v in counter1.items() if counter2[k]>0] # find occurances of the words between the two strings
    

    if len(s)>0: 
        #print(s,stemmed_text2,len(stemmed_text2))
        matching_score = len(s) / len(stemmed_text2)        # calculate the score between skills and normalise it depending on the skill length
    return matching_score

string_match("java","GIAC Secure Software Programmer - Java")

0.16666666666666666

In [55]:
df[df['skill'].str.contains('generat') ==True]

,skill,codes,index
3042,manufacturing of steam generators,[214],3042
3043,steam generators manufacturing,[214],3043
3044,production of steam generators,[214],3044
3049,manufacture of steam generators,[214],3049
3050,fabrication of steam generators,[214],3050
5022,typology of aero generators,[214 741],5022
5023,types of aero generator,[214 741],5023
5024,types of aero-generator,[214 741],5024
5025,kinds of aero generator,[214 741],5025
5332,generation of wooden barrels,[752],5332


In [12]:
from iteration_utilities import duplicates

skills = []
classes = [251, 243, 214, 522, 242,334]#[121, 214, 216, 241, 242, 243, 251, 311, 411, 432]
occupation_codes = []
confidence = []

for i in range(len(certifications)):
    codes = []
    cert_title =  certifications['cert_name'][i]
    extracted_skills = wextract(cert_title)

    if extracted_skills.empty: continue

    for extracted_skill in extracted_skills.skill:
        skill_codes = []
        for taxonomy_skill in range(len(df)):
            if string_match(df['skill'][taxonomy_skill], extracted_skill)>0 : 
                codes_as_string = df['codes'][taxonomy_skill]        # codes are stored as lists of strings
                codes_as_string = codes_as_string.replace("[","")   
                codes_as_string = codes_as_string.replace("]","")
                codes_as_string = codes_as_string.split()
                for code in codes_as_string:
                    if int(code) in classes and int(code) not in skill_codes:               # check whether code belongs at the desired classes 
                        skill_codes.append(int(code))
                        skills.append(df['skill'][taxonomy_skill])
                        #print(df['skill'][taxonomy_skill],code)
        codes.append(skill_codes)

    codes = [item for sublist in codes for item in sublist]         # flatten the list of codes
    if list(duplicates(codes)):                                     # if there are duplicates, it means more than one skill were matched to the same code
        codes = list(duplicates(codes))
        conf = 1
    elif len(codes)<4 and len(codes)>0: 
        conf = 0.5
    else:
        conf = 0
        codes = []
    
    occupation_codes.append(codes)
    confidence.append(conf)

print(occupation_codes,confidence)


[[], [], [251], [], [251], [251], [251], [251], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [251], [334, 243], [], [334], [251], [251], [251], [], [], [], [], [], [251], [251], [251], [214, 243, 522, 242, 334, 251], [334, 243, 251], [], [214, 243, 251], [], [251], [], [], [], [214, 251], [], [251], [], [251], [243, 214], [], [], [], [], [], [], [], [], [], [], [214, 243, 251], [], [], [], [], [], [], [], [], [], [], [], [], [214], [], [], [], [], [], [], [], [], [], [214], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [214, 243], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [251], [251], [251], [], [], [], [], [251], [], [], [251], [], [251], [], [], [], [], [], [], [], [], [], [], [], [251], [], [], [], [], [], [], [], [], [], [], [251], [], [251], [], [], [], [], [], [], [251], [334, 243], [251], [], [], [243, 251], [], [], [], [], [251], [251], [251], [], [], [], [], [], [214, 522], [], [], [], [], [], [], [], 

In [13]:
certifications['prediction'] = occupation_codes
certifications['confidence'] = confidence

In [35]:
j = 0
k=0
for i in range(len(certifications)):
    if certifications['confidence'][i]!=0:
        k+=1
        if certifications.isco3[i] in certifications.prediction[i] : j+=1
print(k,j)

101 74


In [56]:
certifications[certifications['confidence']==0]

In [53]:
certifications.iloc[20:30]

,cert_name,isco3
1327,WSO - Certified Safety Specialist,214
1328,Certified Mine Safety Professional,214
1329,"Diplomate, Geotechnical Engineering",214
1330,WSO - Certified Governmental Environmental Spe...,214
1331,Distributed Generation Certified Professional,214
1332,ANDE Personnel Certification,214
1333,Certified Reliability Engineer,214
1334,Certified Professional Agronomist,214
1335,Certified Reliability Engineer,214
1336,Certified Coal Geologist,311


In [125]:
matched_code = []
index = []

for i in range(len(certifications)):
    cert_title = certifications['cert_name'][i]
    extracted_skills = wextract(cert_title)
    if not extracted_skills.empty:
        for skill in extracted_skills.skill:
            if match_skill(skill) == 0 : 
                continue   # if the skill is not matched continue
            else: 
                matched_code.append(match_skill(skill)) # store only matched skills
                index.append(i)
            

In [137]:
df = pd.DataFrame({'index':index, 'code':matched_code})

df

,index,code
0,256,[311 325 331 241 333 122 121 342 132 242]
1,301,[241 112 122 431 134 333 121 143 331 334 232 2...
2,356,[231 251 311]
3,393,[231 251 311]
4,395,[231 251 311]
5,432,[214 134 132 121 122 251 216 243 235 213 242 2...
6,476,[732 311 312]
7,528,[243 133 242 431 121 241 251]
8,533,[211 213 216 11 21 232 311 315 263 214 31 134]
9,535,[311]


In [123]:
certifications['cert_name'][395]

'GIAC Secure Software Programmer - Java'

In [35]:
import pandas as pd

skills = pd.read_excel('Skills_Occupations Matrix Tables.xlsx', 'Matrix 3.3', header = 1)
skills.rename(columns={'Unnamed: 0': 'escoe','Unnamed: 1': 'Occupations'}, inplace=True)
skills.escoe= skills.escoe.astype(str).str[-3:]
skills.head()

,escoe,Occupations,"communication, collaboration and creativity",negotiating,negotiating and managing contracts and agreements,mediating and resolving disputes,responding to complaints,designing systems and products,designing ict systems or applications,designing electrical or electronic systems or equipment,...,operating communications equipment,operating precision industrial equipment,using precision measuring equipment,"installing, maintaining and repairing mechanical equipment",repairing and installing mechanical equipment,maintaining mechanical equipment,"installing, maintaining and repairing electrical, electronic and precision equipment","installing and repairing electrical, electronic and precision equipment","maintaining electrical, electronic and precision equipment",operating aircraft
0,011,Commissioned armed forces officers,0.000000,0.0,0.001832,0.000000,0.000000,0.000000,0.0,0.000000,...,0.014652,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.003663
1,021,Non-commissioned armed forces officers,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.036842,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,031,"Armed forces occupations, other ranks",0.000000,0.0,0.000000,0.004219,0.000000,0.004219,0.0,0.008439,...,0.092827,0.0,0.004219,0.0,0.004219,0.008439,0.0,0.0,0.012658,0.025316
3,111,Legislators and senior officials,0.002703,0.0,0.005405,0.055405,0.001351,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,112,Managing directors and chief executives,0.000000,0.0,0.035714,0.018797,0.003759,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.001880,0.0,0.0,0.001880,0.000000
